In [18]:
# Core and data handling
import numpy as np
import pandas as pd
import os
from statistics import stdev

# Visualization

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.utils import compute_class_weight
from sklearn.metrics import (
    confusion_matrix, mean_squared_error, precision_recall_curve,
    precision_score, recall_score, accuracy_score, f1_score,
    roc_auc_score, average_precision_score
)

# Imbalanced learning
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
import lightgbm as lgb

# XGBoost
import xgboost as xgb

# Deep Learning
import torch
import torchvision
import torchvision.utils
import tensorflow as tf

# Dataset
from sklearn.datasets import make_classification

# SVM
from sklearn.svm import SVC
## Functions defined by me 
from tqdm import tqdm
# Check GPU availability
print(f"PyTorch CUDA available: {torch.cuda.is_available()}")

# If you want to drop duplicates in a pandas DataFrame, you would do:
# df = df.drop_duplicates()

PyTorch CUDA available: True


## Read Data

In [19]:


from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
default_of_credit_card_clients = fetch_ucirepo(id=350) 
  
# data (as pandas dataframes) 
X = default_of_credit_card_clients.data.features 
y = default_of_credit_card_clients.data.targets 




y=y['Y']

#X.drop(['X1','X2','X3'],axis=1,inplace=True)




# 1️⃣ Concatenate X and y for easy row handling
data = X.copy()
data['target'] = y

# 2️⃣ Find minority class rows (where y == 1)
minority_class = data[data['target'] == 1]

# 3️⃣ Randomly select 10% of them to drop
drop_frac = 0.85  # 10%
n_drop = int(len(minority_class) * drop_frac)
drop_indices = minority_class.sample(n=n_drop, random_state=42).index

# 4️⃣ Drop those rows
data_undersampled = data.drop(index=drop_indices)

# 5️⃣ Separate back into X and y
X = data_undersampled.drop(columns=['target'])
y = data_undersampled['target']

print("Original minority count:", len(minority_class))
print("Remaining minority count:", (y == 1).sum())


Original minority count: 6636
Remaining minority count: 996


In [20]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

# Initialize scaler
scaler = StandardScaler()



# Fit only on training data!
scaler.fit(X_train)
# Example dataset



# Transform training data
X_train = scaler.transform(X_train)

# Transform test data using same scaler (mean & std)
X_test = scaler.transform(X_test)


In [21]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    precision_recall_curve, accuracy_score, roc_auc_score, 
    average_precision_score, recall_score, f1_score, confusion_matrix
)
import pandas as pd
import numpy as np

def calculate_metrics(y_test, y_pred_proba):
    """Calculate binary classification metrics given true labels and predicted probabilities."""
    y_pred_binary = [1 if p > 0.5 else 0 for p in y_pred_proba]

    accuracy = accuracy_score(y_test, y_pred_binary)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    pr_auc = average_precision_score(y_test, y_pred_proba)
    recall = recall_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    fp_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
    g_mean = np.sqrt(recall * specificity)

    metrics_dict = {
        'accuracy': accuracy,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'recall': recall,
        'f1': f1,
        'specificity': specificity,
        'fp_rate': fp_rate,
        'g_mean': g_mean
    }

    return metrics_dict

def callf1(X_train, y_train, X_test, y_test):
    results = {}

    ### ------------------------------
    ### LightGBM
    ### ------------------------------
    dtrain = lgb.Dataset(X_train, label=y_train)
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "device": "gpu"
    }
    model = lgb.train(params, dtrain)
    y_pred_proba = model.predict(X_test)
    results['LightGBM'] = calculate_metrics(y_test, y_pred_proba)

    ### ------------------------------
    ### XGBoost RF
    ### ------------------------------
    rf_model = xgb.XGBClassifier(
        tree_method="gpu_hist",
        predictor="gpu_predictor",
        objective="binary:logistic",
        booster="gbtree",
        num_parallel_tree=100,
        subsample=0.8,
        colsample_bytree=0.8,
        n_estimators=100
    )
    rf_model.fit(X_train, y_train)
    y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]
    results['XGBoost_RF'] = calculate_metrics(y_test, y_pred_proba_rf)

    ### ------------------------------
    ### XGBoost GBDT
    ### ------------------------------
    xgb_model = xgb.XGBClassifier(
        tree_method="gpu_hist",
        predictor="gpu_predictor",
        objective="binary:logistic",
        booster="gbtree",
        n_estimators=100
    )
    xgb_model.fit(X_train, y_train)
    y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
    results['XGBoost_GBDT'] = calculate_metrics(y_test, y_pred_proba_xgb)

    ### ------------------------------
    ### Logistic Regression
    ### ------------------------------
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred_proba_lr = lr.predict_proba(X_test)[:, 1]
    results['LogisticRegression'] = calculate_metrics(y_test, y_pred_proba_lr)

    ### ------------------------------
    ### Combine as DataFrame
    ### ------------------------------
    results_df = pd.DataFrame(results)
    results_df.insert(0, 'metric', results_df.index)
    results_df.reset_index(drop=True, inplace=True)

    return results_df


# 1. Modelling with LightGBM algorithm with just using SMOTE to oversample train data 
## For each configuration, model is trained for 20 times and the averages of the output scores are taken.

In [22]:
# Define an SVM with RBF kernel
#svm_estimator = SVC(kernel='rbf', gamma='scale', random_state=11)

# Apply SVM-SMOTE with the RBF kernel SVM
svm_smote = SMOTE(
    #svm_estimator=svm_estimator,  # Use the RBF kernel SVM
    random_state=11
)
X_train_resampled, y_train_resampled = svm_smote.fit_resample(X_train, y_train)

In [23]:
from tqdm import tqdm

In [24]:
sum=0
for i in range(1):
    r=callf1(X_train_resampled , y_train_resampled.ravel(),X_test,y_test.ravel())

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_3364\3781152710.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_train_resampled , y_train_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 16355, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5830
[LightGBM] [Info] Number of data points in the train set: 32710, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.75 MB) transferred to GPU in 0.006345 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:36:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [25]:
for i in range(1):
    rson=callf1(X_train , y_train.ravel(),X_test,y_test.ravel())

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_3364\1221683945.py:2: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rson=callf1(X_train , y_train.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 697, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3262
[LightGBM] [Info] Number of data points in the train set: 17052, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.39 MB) transferred to GPU in 0.004169 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.040875 -> initscore=-3.155504
[LightGBM] [Info] Start training from score -3.155504


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:36:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:36:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:36:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

[LightGBM] [Info] Number of positive: 697, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3262
[LightGBM] [Info] Number of data points in the train set: 17052, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.39 MB) transferred to GPU in 0.003732 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.040875 -> initscore=-3.155504
[LightGBM] [Info] Start training from score -3.155504


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:36:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:36:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:37:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

[LightGBM] [Info] Number of positive: 697, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3262
[LightGBM] [Info] Number of data points in the train set: 17052, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.39 MB) transferred to GPU in 0.003881 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.040875 -> initscore=-3.155504
[LightGBM] [Info] Start training from score -3.155504


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:37:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:37:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)

KeyboardInterrupt



In [27]:
r

,metric,LightGBM,XGBoost_RF,XGBoost_GBDT,LogisticRegression
0,accuracy,0.952928,0.955665,0.953202,0.673098
1,roc_auc,0.726939,0.713902,0.688268,0.721412
2,pr_auc,0.176495,0.168471,0.145518,0.163311
3,recall,0.150502,0.133779,0.127090,0.645485
4,f1,0.207373,0.198020,0.181818,0.139099
5,specificity,0.987159,0.990726,0.988443,0.674276
6,fp_rate,0.012841,0.009274,0.011557,0.325724
7,g_mean,0.385447,0.364059,0.354431,0.659723


In [26]:
rson

,metric,LightGBM,XGBoost_RF,XGBoost_GBDT,LogisticRegression
0,accuracy,0.956486,0.957991,0.956486,0.958539
1,roc_auc,0.756872,0.738574,0.733228,0.711557
2,pr_auc,0.169143,0.182613,0.142699,0.168823
3,recall,0.033445,0.043478,0.053512,0.010033
4,f1,0.059172,0.078078,0.091429,0.019417
5,specificity,0.995862,0.997004,0.995006,0.999001
6,fp_rate,0.004138,0.002996,0.004994,0.000999
7,g_mean,0.182501,0.208202,0.230748,0.100117


In [28]:
import numpy as np
import torch
from torch import nn, autograd
from torch.utils.data import TensorDataset, DataLoader
from tqdm.auto import tqdm
from imblearn.under_sampling import EditedNearestNeighbours
import shap
from lightgbm import LGBMClassifier

# ==== Generator Block ====
def get_generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )

class ConditionalGenerator(nn.Module):
    def __init__(self, z_dim, label_dim, im_dim, hidden_dim=128):
        super().__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim + label_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
        )

    def forward(self, noise, labels):
        input = torch.cat((noise, labels), dim=1)
        return self.gen(input)

# ==== Discriminator / Critic ====
class ConditionalCritic(nn.Module):
    def __init__(self, im_dim, label_dim, hidden_dim=128):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(im_dim + label_dim, hidden_dim * 4),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim * 4, hidden_dim * 2),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x, labels):
        input = torch.cat((x, labels), dim=1)
        return self.disc(input)

# ==== Gradient Penalty ====
def gradient_penalty(critic, real_data, fake_data, labels, device, lambda_gp=10):
    batch_size = real_data.size(0)
    alpha = torch.rand(batch_size, 1).to(device).expand_as(real_data)
    interpolated = (alpha * real_data + (1 - alpha) * fake_data).requires_grad_(True)
    interpolated_labels = labels.to(device)
    critic_interpolated = critic(interpolated, interpolated_labels)
    gradients = autograd.grad(
        outputs=critic_interpolated,
        inputs=interpolated,
        grad_outputs=torch.ones_like(critic_interpolated),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradients = gradients.view(batch_size, -1)
    return ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lambda_gp

# ==== Shuffle utility ====
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

# ==== WGAN-GP with SHAP-weighted latent noise ====
def wgan_gp_shap_weighted(
    X_train,
    y_train,
    apply_enn=True,
    n_epochs=200,
    batch_size=128,
    critic_iterations=5,
    lr=1e-4,
    n_synthetic_factor=2
):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input_dim = X_train.shape[1]

    # --- Train LightGBM to get SHAP importances for minority class ---
    lgb = LGBMClassifier(n_estimators=300)
    lgb.fit(X_train, y_train)
    
    explainer = shap.TreeExplainer(lgb)
    shap_values = explainer.shap_values(X_train)  # list of arrays per class
    # Take absolute mean of SHAP values for minority class (assume class 1)
    minority_shap = np.abs(shap_values[1]).mean(axis=0)
    feature_weights = minority_shap / minority_shap.sum()  # normalized
    print('Shap Calculated.')

    # --- Use only real minority class samples ---
    y_tr = y_train.ravel()
    X_real = np.array([X_train[i] for i in range(len(y_tr)) if int(y_tr[i]) == 1])
    y_real = np.ones(len(X_real))

    X_minority = np.array([X_train[i] for i in range(len(y_tr)) if int(y_tr[i]) == 1])
    n_minority = len(X_minority)
    n_majority = np.sum(y_tr == 0)
    n_generate_1 = n_majority - n_minority  # exactly balance classes
    
    tensor_x = torch.Tensor(X_real)
    tensor_y = torch.Tensor(y_real)
    dataset = TensorDataset(tensor_x, tensor_y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    label_dim = 1
    gen = ConditionalGenerator(z_dim=input_dim, label_dim=label_dim, im_dim=input_dim).to(device)
    critic = ConditionalCritic(im_dim=input_dim, label_dim=label_dim).to(device)

    gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(0.0, 0.9))
    critic_opt = torch.optim.Adam(critic.parameters(), lr=lr, betas=(0.0, 0.9))

    # === Train WGAN-GP ===
    for epoch in range(n_epochs):
        for real, _ in tqdm(dataloader, leave=False):
            real = real.to(device)
            real_labels = torch.ones((real.size(0), 1)).to(device)

            for _ in range(critic_iterations):
                # --- Latent noise scaled by SHAP weights ---
                z = torch.randn(real.size(0), input_dim).to(device)
                z = z * torch.tensor(np.sqrt(feature_weights), device=device)
                noise_labels = torch.ones((real.size(0), 1)).to(device)
                fake = gen(z, noise_labels)

                critic_real = critic(real, real_labels).mean()
                critic_fake = critic(fake.detach(), noise_labels).mean()
                gp = gradient_penalty(critic, real, fake.detach(), real_labels, device)

                critic_loss = -(critic_real - critic_fake) + gp
                critic_opt.zero_grad()
                critic_loss.backward()
                critic_opt.step()

            # === Update Generator ===
            z = torch.randn(real.size(0), input_dim).to(device)
            z = z * torch.tensor(np.sqrt(feature_weights), device=device)
            noise_labels = torch.ones((real.size(0), 1)).to(device)
            gen_fake = gen(z, noise_labels)
            gen_loss = -critic(gen_fake, noise_labels).mean()

            gen_opt.zero_grad()
            gen_loss.backward()
            gen_opt.step()

        print(f"Epoch {epoch} | Gen Loss: {gen_loss.item():.4f}, Critic Loss: {critic_loss.item():.4f}")

    # === Generate synthetic samples using weighted latent noise ---
    with torch.no_grad():
        n_generate = n_generate_1
        z = torch.randn(n_generate, input_dim).to(device)
        z = z * torch.tensor(np.sqrt(feature_weights), device=device)
        noise_labels = torch.ones((n_generate, 1)).to(device)
        generated = gen(z, noise_labels).cpu().numpy()
 

    # === Combine real and synthetic data ===
    final_data = np.concatenate((X_train, generated), axis=0)
    final_labels = np.concatenate((y_train, np.ones(len(generated))), axis=0)
    X_out, y_out = shuffle_in_unison(final_data, final_labels)


    return X_out, y_out


In [43]:
import numpy as np
import torch
from torch import nn, autograd
from torch.utils.data import TensorDataset, DataLoader
from tqdm.auto import tqdm
from imblearn.under_sampling import EditedNearestNeighbours
import shap
from lightgbm import LGBMClassifier

# ==== Generator Block ====
def get_generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )

class ConditionalGenerator(nn.Module):
    def __init__(self, z_dim, label_dim, im_dim, hidden_dim=128):
        super().__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim + label_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
        )

    def forward(self, noise, labels):
        input = torch.cat((noise, labels), dim=1)
        return self.gen(input)

# ==== Discriminator / Critic ====
class ConditionalCritic(nn.Module):
    def __init__(self, im_dim, label_dim, hidden_dim=128):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(im_dim + label_dim, hidden_dim * 4),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim * 4, hidden_dim * 2),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x, labels):
        input = torch.cat((x, labels), dim=1)
        return self.disc(input)

# ==== Gradient Penalty ====
def gradient_penalty(critic, real_data, fake_data, labels, device, lambda_gp=10):
    batch_size = real_data.size(0)
    alpha = torch.rand(batch_size, 1).to(device).expand_as(real_data)
    interpolated = (alpha * real_data + (1 - alpha) * fake_data).requires_grad_(True)
    interpolated_labels = labels.to(device)
    critic_interpolated = critic(interpolated, interpolated_labels)
    gradients = autograd.grad(
        outputs=critic_interpolated,
        inputs=interpolated,
        grad_outputs=torch.ones_like(critic_interpolated),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradients = gradients.view(batch_size, -1)
    return ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lambda_gp

# ==== Shuffle utility ====
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

# === WGAN-GP with SOFT SHAP-weighted latent + feature matching ===
def wgan_gp_shap_weighted_soft(
    X_train,
    y_train,
    apply_enn=True,
    n_epochs=300,
    batch_size=128,
    critic_iterations=5,
    lr=1e-4,
    n_synthetic_factor=2,
    alpha=0.5,       # blend between isotropic and shap-weighted latent
    beta=0.1         # weight for feature-matching penalty
):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input_dim = X_train.shape[1]

    # --- Train LightGBM to get SHAP importances for minority class ---
    lgb = LGBMClassifier(n_estimators=300)
    lgb.fit(X_train, y_train)
    
    explainer = shap.TreeExplainer(lgb)
    shap_values = explainer.shap_values(X_train)  # list of arrays per class
    minority_shap = np.abs(shap_values[1]).mean(axis=0)
    feature_weights = minority_shap / minority_shap.sum()
    print('Shap Calculated.')

    # Convert weights to torch tensor once
    shap_tensor = torch.tensor(feature_weights, dtype=torch.float32, device=device)

    # --- Use only minority class samples ---
    y_tr = y_train.ravel()
    X_real = np.array([X_train[i] for i in range(len(y_tr)) if int(y_tr[i]) == 1])
    y_real = np.ones(len(X_real))

    n_minority = len(X_real)
    n_majority = np.sum(y_tr == 0)
    n_generate_1 = n_majority - n_minority  # exactly balance classes
    
    tensor_x = torch.Tensor(X_real)
    tensor_y = torch.Tensor(y_real)
    dataset = TensorDataset(tensor_x, tensor_y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    label_dim = 1
    gen = ConditionalGenerator(z_dim=input_dim, label_dim=label_dim, im_dim=input_dim).to(device)
    critic = ConditionalCritic(im_dim=input_dim, label_dim=label_dim).to(device)

    gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(0.0, 0.9))
    critic_opt = torch.optim.Adam(critic.parameters(), lr=lr, betas=(0.0, 0.9))

    # === Train WGAN-GP ===
    for epoch in range(n_epochs):
        for real, _ in tqdm(dataloader, leave=False):
            real = real.to(device)
            real_labels = torch.ones((real.size(0), 1)).to(device)

            for _ in range(critic_iterations):
                # --- Soft SHAP-weighted latent ---
                z_iso = torch.randn(real.size(0), input_dim, device=device)
                z_shap = z_iso * shap_tensor.sqrt()
                z = alpha * z_shap + (1 - alpha) * z_iso

                noise_labels = torch.ones((real.size(0), 1), device=device)
                fake = gen(z, noise_labels)

                critic_real = critic(real, real_labels).mean()
                critic_fake = critic(fake.detach(), noise_labels).mean()
                gp = gradient_penalty(critic, real, fake.detach(), real_labels, device)

                critic_loss = -(critic_real - critic_fake) + gp
                critic_opt.zero_grad()
                critic_loss.backward()
                critic_opt.step()

            # === Update Generator ===
            z_iso = torch.randn(real.size(0), input_dim, device=device)
            z_shap = z_iso * shap_tensor.sqrt()
            z = alpha * z_shap + (1 - alpha) * z_iso

            noise_labels = torch.ones((real.size(0), 1), device=device)
            gen_fake = gen(z, noise_labels)

            # Adversarial loss
            gen_loss = -critic(gen_fake, noise_labels).mean()

            # Feature-matching penalty (SHAP-weighted L1 distance)
            fm_loss = (torch.abs(real[:gen_fake.size(0)] - gen_fake) * shap_tensor).mean()
            gen_loss = gen_loss + beta * fm_loss

            gen_opt.zero_grad()
            gen_loss.backward()
            gen_opt.step()

        print(f"Epoch {epoch} | Gen Loss: {gen_loss.item():.4f}, Critic Loss: {critic_loss.item():.4f}")

    # === Generate synthetic samples using soft SHAP-weighted latent ---
    with torch.no_grad():
        n_generate = n_generate_1
        z_iso = torch.randn(n_generate, input_dim, device=device)
        z_shap = z_iso * shap_tensor.sqrt()
        z = alpha * z_shap + (1 - alpha) * z_iso

        noise_labels = torch.ones((n_generate, 1), device=device)
        generated = gen(z, noise_labels).cpu().numpy()
 
    # === Combine real and synthetic data ===
    final_data = np.concatenate((X_train, generated), axis=0)
    final_labels = np.concatenate((y_train, np.ones(len(generated))), axis=0)
    X_out, y_out = shuffle_in_unison(final_data, final_labels)

    return X_out, y_out



In [68]:
# tabular_diffusion_oversampler.py
# -----------------------------------------------------------
# Tabular Diffusion Oversampler (minority-class conditional)
# - Numerical features only
# - Conditioning: class token + SHAP (or gain) importance vector
# - Classifier-free guidance at sampling
# - Post-selection by kNN distance and optional classifier confidence
# - Hardened against NaN/Inf via clamps and sanitization
# -----------------------------------------------------------

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from sklearn.neighbors import NearestNeighbors
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, average_precision_score

# =========================
# Utilities
# =========================

def set_seed(seed: int = 42):
    import random, os
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

def standardize(X, mean=None, std=None, eps=1e-8):
    X = np.asarray(X, dtype=np.float32)
    if mean is None: mean = X.mean(axis=0, keepdims=True).astype(np.float32)
    if std  is None:
        std = X.std(axis=0, keepdims=True).astype(np.float32)
        std = np.where(std < eps, eps, std)
    Xs = (X - mean) / std
    return Xs, mean, std

def unstandardize(Xs, mean, std):
    Xr = Xs * std + mean
    return Xr

def np_sanitize(a, clip=None):
    a = np.nan_to_num(a, nan=0.0, posinf=1e6, neginf=-1e6)
    if clip is not None:
        a = np.clip(a, -clip, clip)
    return a.astype(np.float32, copy=False)

# =========================
# SHAP / Gain importance
# =========================

def minority_importance_vector(X, y, use_gain=False):
    """
    Returns a normalized importance vector (shape [D]) for conditioning.
    If use_gain=True, uses LightGBM gain importance. Else uses SHAP values (minority class).
    Robust to SHAP returning list/tuple for binary classification.
    """
    lgb = LGBMClassifier(n_estimators=300)
    lgb.fit(X, y)

    if use_gain:
        try:
            gains = lgb.booster_.feature_importance(importance_type="gain")
        except Exception:
            gains = lgb.feature_importances_
        imp = np.asarray(gains, dtype=np.float64)
    else:
        import shap
        explainer = shap.TreeExplainer(lgb)
        shap_vals = explainer.shap_values(X)
        if isinstance(shap_vals, (list, tuple)):
            shap_arr = shap_vals[1] if len(shap_vals) >= 2 else shap_vals[0]
        else:
            shap_arr = shap_vals
        imp = np.abs(shap_arr).mean(axis=0).astype(np.float64)

    # sanitize & normalize
    imp = np.nan_to_num(imp, nan=0.0, posinf=0.0, neginf=0.0)
    imp = np.maximum(imp, 0.0)
    s = imp.sum()
    if s <= 0:
        imp = np.ones_like(imp, dtype=np.float64) / len(imp)
    else:
        imp = imp / s
    return imp.astype(np.float32)

# =========================
# Time embeddings (sinusoidal)
# =========================

class SinusoidalTimeEmb(nn.Module):
    def __init__(self, dim: int):
        super().__init__()
        self.dim = dim

    def forward(self, t: torch.Tensor):
        # t: (B,) integer timesteps
        device = t.device
        half = self.dim // 2
        if half == 0:
            return torch.zeros(t.size(0), self.dim, device=device)
        freqs = torch.exp(
            -torch.arange(half, device=device, dtype=torch.float32) * (np.log(10000.0) / max(1, half - 1))
        )
        args = t.float().unsqueeze(1) * freqs.unsqueeze(0)
        emb = torch.cat([torch.sin(args), torch.cos(args)], dim=1)
        if emb.size(1) < self.dim:  # odd dim
            emb = F.pad(emb, (0, 1))
        return emb  # (B, dim)

# =========================
# FiLM block (conditioned MLP)
# =========================

class FiLM(nn.Module):
    def __init__(self, in_dim, cond_dim):
        super().__init__()
        self.gamma = nn.Linear(cond_dim, in_dim)
        self.beta  = nn.Linear(cond_dim, in_dim)

    def forward(self, x, cond):
        g = self.gamma(cond)
        b = self.beta(cond)
        return x * (1 + g) + b

class DenoiserMLP(nn.Module):
    """
    Predicts noise epsilon for x_t given (t, cond).
    cond = [class_token (1-d), importance vector (D)] -> projected to cond_dim
    """
    def __init__(self, data_dim, time_dim=128, cond_dim=128, hidden=512):
        super().__init__()
        self.data_dim = data_dim
        self.time_emb = SinusoidalTimeEmb(time_dim)

        # project conditioning to cond_dim
        self.cond_proj = nn.Sequential(
            nn.Linear(data_dim + 1, cond_dim),
            nn.SiLU(),
            nn.Linear(cond_dim, cond_dim)
        )

        self.fc1 = nn.Linear(data_dim + time_dim, hidden)
        self.bn1 = nn.BatchNorm1d(hidden)
        self.film1 = FiLM(hidden, cond_dim)

        self.fc2 = nn.Linear(hidden, hidden)
        self.bn2 = nn.BatchNorm1d(hidden)
        self.film2 = FiLM(hidden, cond_dim)

        self.fc3 = nn.Linear(hidden, data_dim)

    def forward(self, x, t, cond_vec):
        """
        x: (B, D), t: (B,), cond_vec: (B, D+1)
        """
        te = self.time_emb(t)                          # (B, time_dim)
        h = torch.cat([x, te], dim=1)                  # (B, D+time_dim)
        h = self.fc1(h); h = self.bn1(h); h = F.silu(h)
        c = self.cond_proj(cond_vec)                   # (B, cond_dim)
        h = self.film1(h, c); h = F.silu(h)
        h = self.fc2(h); h = self.bn2(h); h = F.silu(h)
        h = self.film2(h, c); h = F.silu(h)
        out = self.fc3(h)
        return out

# =========================
# Diffusion scheduler
# =========================

@dataclass
class DiffusionConfig:
    timesteps: int = 500
    beta_start: float = 1e-4
    beta_end: float = 0.02
    guidance_p_drop: float = 0.1  # classifier-free guidance dropout prob
    guidance_scale: float = 1.5   # sampling guidance scale
    clip_std: float = 10.0        # clip during sampling in standardized space

class Diffusion:
    def __init__(self, data_dim, cfg: DiffusionConfig):
        self.cfg = cfg
        T = cfg.timesteps
        betas = torch.linspace(cfg.beta_start, cfg.beta_end, T, dtype=torch.float32)
        alphas = 1.0 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        self.register_buffers(betas, alphas, alphas_cumprod)
        self.model = DenoiserMLP(data_dim=data_dim)

    def register_buffers(self, betas, alphas, alphas_cumprod):
        device = get_device()
        self.betas = betas.to(device)                        # (T,)
        self.alphas = alphas.to(device)                      # (T,)
        self.alphas_cumprod = alphas_cumprod.to(device)      # (T,)

        # Precomputes
        self.sqrt_alphas_cumprod = torch.sqrt(self.alphas_cumprod.clamp_min(1e-12))
        self.sqrt_one_minus_alphas_cumprod = torch.sqrt((1.0 - self.alphas_cumprod).clamp_min(1e-12))
        self.one_over_sqrt_alpha = torch.sqrt((1.0 / self.alphas).clamp_min(1e-12))

        # Proper posterior variance (length T):
        alpha_bar_prev = torch.cat(
            [torch.ones(1, device=device), self.alphas_cumprod[:-1]], dim=0
        )                                                                               # (T,)
        denom = (1.0 - self.alphas_cumprod).clamp_min(1e-12)                            # (T,)
        self.posterior_var = (self.betas * (1.0 - alpha_bar_prev) / denom).clamp_min(1e-12)  # (T,)

    def to(self, device):
        self.model = self.model.to(device)

    # q(x_t | x_0)
    def q_sample(self, x0, t, noise):
        return self.sqrt_alphas_cumprod[t].unsqueeze(1) * x0 + \
               self.sqrt_one_minus_alphas_cumprod[t].unsqueeze(1) * noise

    # loss: MSE between predicted noise and true noise
    def loss(self, x0, cond_vec):
        device = x0.device
        B = x0.size(0)
        t = torch.randint(0, self.cfg.timesteps, (B,), device=device)
        noise = torch.randn_like(x0)
        x_t = self.q_sample(x0, t, noise)

        # classifier-free guidance dropout
        drop_mask = (torch.rand(B, device=device) < self.cfg.guidance_p_drop).float().unsqueeze(1)
        null_cond = torch.zeros_like(cond_vec)
        cond_in = cond_vec * (1 - drop_mask) + null_cond * drop_mask

        eps_pred = self.model(x_t, t, cond_in)
        loss = F.mse_loss(eps_pred, noise)

        # NaN guard: replace if necessary
        if torch.isnan(loss):
            loss = torch.tensor(0.0, device=device, requires_grad=True)
        return loss

    @torch.no_grad()
    def p_sample(self, x_t, t_int, cond_vec, null_vec):
        """
        One reverse step at scalar timestep t_int (int).
        Uses classifier-free guidance:
           eps = (1+w)*eps(cond) - w*eps(null)
        """
        device = x_t.device
        B = x_t.size(0)
        t_vec = torch.full((B,), int(t_int), device=device, dtype=torch.long)

        eps_cond = self.model(x_t, t_vec, cond_vec)
        eps_null = self.model(x_t, t_vec, null_vec)
        # sanitize eps
        eps_cond = torch.nan_to_num(eps_cond)
        eps_null = torch.nan_to_num(eps_null)

        w = self.cfg.guidance_scale
        eps = (1 + w) * eps_cond - w * eps_null
        eps = torch.nan_to_num(eps)

        beta_t = self.betas[t_int]                           # scalar
        alpha_t = self.alphas[t_int]                         # scalar
        alpha_bar_t = self.alphas_cumprod[t_int]             # scalar
        one_minus_bar = torch.clamp(1.0 - alpha_bar_t, min=1e-12)

        # predict x0
        x0_pred = (x_t - torch.sqrt(one_minus_bar) * eps) / torch.sqrt(alpha_bar_t).clamp_min(1e-12)
        # mean of p(x_{t-1} | x_t, x0)
        mean = torch.sqrt(alpha_t).clamp_min(1e-12) * x0_pred + torch.sqrt(1.0 - alpha_t).clamp_min(1e-12) * eps

        # clip in standardized space to avoid blow-ups
        if self.cfg.clip_std is not None:
            mean = torch.clamp(mean, -self.cfg.clip_std, self.cfg.clip_std)

        if t_int == 0:
            x_next = mean
        else:
            var = self.posterior_var[t_int]                  # scalar
            noise = torch.randn_like(x_t)
            x_next = mean + torch.sqrt(var).clamp_min(1e-12) * noise

        # sanitize
        x_next = torch.nan_to_num(x_next)
        if self.cfg.clip_std is not None:
            x_next = torch.clamp(x_next, -self.cfg.clip_std, self.cfg.clip_std)
        return x_next

    @torch.no_grad()
    def sample(self, n, cond_vec, device):
        """
        cond_vec: (n, D+1) tensor on device
        """
        D = cond_vec.size(1) - 1
        x_t = torch.randn(n, D, device=device)
        null_vec = torch.zeros_like(cond_vec)

        for t in reversed(range(self.cfg.timesteps)):
            x_t = self.p_sample(x_t, t, cond_vec, null_vec)
        return x_t

# =========================
# Training & Sampling API
# =========================

def build_cond_matrix(num_rows, imp_vec, device):
    """
    Build conditioning matrix: [class_token(=1), imp_vec]
    Returns (num_rows, D+1) tensor.
    """
    imp_vec = np_sanitize(imp_vec)
    imp = torch.tensor(imp_vec, dtype=torch.float32, device=device).unsqueeze(0).repeat(num_rows, 1)
    cls = torch.ones(num_rows, 1, device=device)  # minority token
    cond = torch.cat([cls, imp], dim=1)
    cond = torch.nan_to_num(cond)
    return cond

def train_diffusion_minority(X, y, use_gain=False, epochs=400, batch_size=256, lr=2e-4, seed=42):
    """
    Train diffusion model on standardized minority samples only.
    Returns: trained Diffusion object, and (mean,std) for de-standardization.
    """
    set_seed(seed)
    device = get_device()

    # standardize on the minority distribution (safer)
    X = np.asarray(X, dtype=np.float32)
    y = np.asarray(y).reshape(-1)
    X_min = X[y == 1]
    if len(X_min) < 10:
        raise ValueError("Too few minority samples to train diffusion.")

    X_min_std, mu, sigma = standardize(X_min)

    imp_vec = minority_importance_vector(X, y, use_gain=use_gain)  # interpretable prior

    D = X.shape[1]
    cfg = DiffusionConfig()
    diff = Diffusion(D, cfg)
    diff.to(device)

    opt = torch.optim.Adam(diff.model.parameters(), lr=lr)
    X_min_t = torch.tensor(np_sanitize(X_min_std, clip=cfg.clip_std), dtype=torch.float32, device=device)

    steps_per_epoch = int(np.ceil(len(X_min_t) / batch_size))
    for ep in range(epochs):
        perm = torch.randperm(len(X_min_t), device=device)
        for step in range(steps_per_epoch):
            idx = perm[step*batch_size:(step+1)*batch_size]
            xb = X_min_t[idx]
            cond = build_cond_matrix(xb.size(0), imp_vec, device)
            loss = diff.loss(xb, cond)
            opt.zero_grad()
            loss.backward()
            opt.step()
        if (ep+1) % 50 == 0 or ep == 0:
            print(f"[Diffusion] Epoch {ep+1}/{epochs} | Loss: {loss.item():.4f}")

    return diff, (mu, sigma), imp_vec

@torch.no_grad()
def generate_minority(diff, mu_sigma, imp_vec, n_samples):
    """
    Sample n_samples synthetic minority in original (unstandardized) space.
    """
    device = get_device()
    mu, sigma = mu_sigma
    cond = build_cond_matrix(n_samples, imp_vec, device)
    xs = diff.sample(n_samples, cond, device=device)  # standardized space
    xs = torch.nan_to_num(xs)
    if diff.cfg.clip_std is not None:
        xs = torch.clamp(xs, -diff.cfg.clip_std, diff.cfg.clip_std)
    xs = xs.cpu().numpy()
    xs = unstandardize(xs, mu, sigma)
    xs = np_sanitize(xs, clip=1e6)
    return xs

# =========================
# Post-selection
# =========================

def postselect_knn(X_gen, X_min, keep_quantile=0.6, k=5):
    """
    Keep the closest fraction (by mean kNN distance to real minority).
    """
    if len(X_gen) == 0:
        return X_gen
    X_gen = np_sanitize(X_gen)     # <- guard
    X_min = np_sanitize(X_min)     # <- guard
    k = min(k, len(X_min))
    nbrs = NearestNeighbors(n_neighbors=k).fit(X_min)
    dists, _ = nbrs.kneighbors(X_gen)
    mean_d = dists.mean(axis=1)
    cutoff = np.quantile(mean_d, keep_quantile)
    return X_gen[mean_d <= cutoff]

def postselect_by_classifier(X_train, y_train, X_candidates, keep_quantile=0.5):
    """
    Train a quick LGBM on (X_train,y_train), keep candidates with top minority probabilities.
    """
    if len(X_candidates) == 0:
        return X_candidates
    X_candidates = np_sanitize(X_candidates)
    clf = LGBMClassifier(n_estimators=250)
    clf.fit(X_train, y_train)
    proba = clf.predict_proba(X_candidates)[:, 1]
    thr = np.quantile(proba, keep_quantile)
    return X_candidates[proba >= thr]

# =========================
# Full pipeline
# =========================

def tabular_diffusion_oversample_pipeline(
    X, y,
    n_generate_factor=1.0,     # how many synthetic vs minority count (e.g. 1.0 = same as #minority)
    use_gain=False,            # True to avoid SHAP overhead
    train_epochs=400,
    batch_size=256,
    lr=2e-4,
    guidance_scale=1.5,
    guidance_pdrop=0.1,
    post_knn_quantile=0.6,
    post_clf_quantile=None,    # set like 0.5 to also apply classifier confidence filter
    seed=42
):
    """
    Returns X_aug, y_aug. Oversamples only y==1 via diffusion.
    """
    set_seed(seed)
    X = np.asarray(X, dtype=np.float32)
    y = np.asarray(y).reshape(-1)
    device = get_device()

    # Train diffusion on minority
    diff, mu_sigma, imp_vec = train_diffusion_minority(
        X, y, use_gain=use_gain, epochs=train_epochs, batch_size=batch_size, lr=lr, seed=seed
    )

    # adjust guidance
    diff.cfg.guidance_scale = guidance_scale
    diff.cfg.guidance_p_drop = guidance_pdrop

    n_min = int((y == 1).sum())
    n_gen = max(1, int(n_min * n_generate_factor))
    X_gen = generate_minority(diff, mu_sigma, imp_vec, n_gen)

    # Post-selection
    X_min = X[y == 1]
    X_sel = postselect_knn(X_gen, X_min, keep_quantile=post_knn_quantile, k=5)

    if (post_clf_quantile is not None) and (len(X_sel) > 0):
        X_sel = postselect_by_classifier(X, y, X_sel, keep_quantile=post_clf_quantile)

    # Combine
    X_out = np.vstack([X, X_sel])
    y_out = np.hstack([y, np.ones(len(X_sel), dtype=int)])
    return X_out, y_out

# =========================
# Quick evaluation helper
# =========================

def evaluate_downstream(X_train_aug, y_train_aug, X_test, y_test):
    clf = LGBMClassifier(n_estimators=400)
    clf.fit(X_train_aug, y_train_aug)
    preds = clf.predict(X_test)
    proba = clf.predict_proba(X_test)[:, 1]
    f1 = f1_score(y_test, preds, pos_label=1)
    ap = average_precision_score(y_test, proba)
    return {'f1_minority': float(f1), 'avg_precision': float(ap)}


In [69]:
X_aug, y_aug = tabular_diffusion_oversample_pipeline(
    X_train, y_train,
    n_generate_factor= (y_train==0).sum()/(y_train==1).sum() - 1.0,  # roughly balance
    use_gain=False,            # if SHAP is slow in your env, set True
    train_epochs=400,          # try 300–600
    batch_size=256,
    lr=2e-4,
    guidance_scale=1.5,        # 1.0–2.5
    guidance_pdrop=0.1,        # 0.1–0.2
    post_knn_quantile=0.6,     # keep closest 60%
    post_clf_quantile=0.5,     # optionally keep top-50% by classifier confidence
    seed=42
)

[LightGBM] [Info] Number of positive: 697, number of negative: 16355
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002601 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3262
[LightGBM] [Info] Number of data points in the train set: 17052, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.040875 -> initscore=-3.155504
[LightGBM] [Info] Start training from score -3.155504


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\shap\explainers\_tree.py:583: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


[Diffusion] Epoch 1/400 | Loss: 0.9973
[Diffusion] Epoch 50/400 | Loss: 0.3238
[Diffusion] Epoch 100/400 | Loss: 0.2768
[Diffusion] Epoch 150/400 | Loss: 0.2741
[Diffusion] Epoch 200/400 | Loss: 0.2749
[Diffusion] Epoch 250/400 | Loss: 0.2545
[Diffusion] Epoch 300/400 | Loss: 0.2477
[Diffusion] Epoch 350/400 | Loss: 0.2316
[Diffusion] Epoch 400/400 | Loss: 0.2369
[LightGBM] [Info] Number of positive: 697, number of negative: 16355
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002609 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3262
[LightGBM] [Info] Number of data points in the train set: 17052, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.040875 -> initscore=-3.155504
[LightGBM] [Info] Start training from score -3.155504


In [79]:
# 🔀 Shuffle output
perm = np.random.permutation(len(y_aug))
X_out = X_aug[perm]
y_out = y_aug[perm]

In [81]:
for i in range(1):
    rnew3=callf1(X_out,y_out.ravel(),X_test,y_test.ravel())

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_3364\1563891602.py:2: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rnew3=callf1(X_out,y_out.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 5395, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5860
[LightGBM] [Info] Number of data points in the train set: 21750, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.50 MB) transferred to GPU in 0.008511 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.248046 -> initscore=-1.109061
[LightGBM] [Info] Start training from score -1.109061


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:15:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:15:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:16:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [44]:
X_res2,y_res2=wgan_gp_shap_weighted_soft(X_train,y_train)

[LightGBM] [Info] Number of positive: 697, number of negative: 16355
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002269 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3262
[LightGBM] [Info] Number of data points in the train set: 17052, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.040875 -> initscore=-3.155504
[LightGBM] [Info] Start training from score -3.155504


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\shap\explainers\_tree.py:583: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_3364\296556924.py:103: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_tr = y_train.ravel()


Shap Calculated.


Epoch 0 | Gen Loss: -0.4809, Critic Loss: 1.0295


Epoch 1 | Gen Loss: -1.2112, Critic Loss: -0.7510


...

Epoch 295 | Gen Loss: 2.1398, Critic Loss: -1.2109


Epoch 296 | Gen Loss: 2.1990, Critic Loss: -1.3054


Epoch 297 | Gen Loss: 1.7647, Critic Loss: -1.5770


Epoch 298 | Gen Loss: 1.6856, Critic Loss: -1.2546


Epoch 299 | Gen Loss: 2.0078, Critic Loss: -1.3427


In [45]:
for i in range(1):
    rnew2=callf1(X_res2,y_res2.ravel(),X_test,y_test.ravel())

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_3364\3974904658.py:2: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rnew2=callf1(X_res2,y_res2.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 16355, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5865
[LightGBM] [Info] Number of data points in the train set: 32710, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.75 MB) transferred to GPU in 0.011190 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:03:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:03:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:04:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [46]:
rnew2

,metric,LightGBM,XGBoost_RF,XGBoost_GBDT,LogisticRegression
0,accuracy,0.957581,0.957718,0.957033,0.688013
1,roc_auc,0.760821,0.743090,0.730022,0.724829
2,pr_auc,0.172646,0.181008,0.159587,0.154778
3,recall,0.023411,0.046823,0.063545,0.645485
4,f1,0.043210,0.083086,0.107955,0.144786
5,specificity,0.997432,0.996576,0.995149,0.689827
6,fp_rate,0.002568,0.003424,0.004851,0.310173
7,g_mean,0.152811,0.216015,0.251469,0.667288


In [42]:
rson

,metric,LightGBM,XGBoost_RF,XGBoost_GBDT,LogisticRegression
0,accuracy,0.956486,0.957991,0.956486,0.958539
1,roc_auc,0.756872,0.738574,0.733228,0.711557
2,pr_auc,0.169143,0.182613,0.142699,0.168823
3,recall,0.033445,0.043478,0.053512,0.010033
4,f1,0.059172,0.078078,0.091429,0.019417
5,specificity,0.995862,0.997004,0.995006,0.999001
6,fp_rate,0.004138,0.002996,0.004994,0.000999
7,g_mean,0.182501,0.208202,0.230748,0.100117


In [40]:
rnew2

,metric,LightGBM,XGBoost_RF,XGBoost_GBDT,LogisticRegression
0,accuracy,0.957170,0.957718,0.957170,0.697181
1,roc_auc,0.764943,0.742188,0.733892,0.724531
2,pr_auc,0.180318,0.185911,0.158892,0.156020
3,recall,0.020067,0.040134,0.053512,0.632107
4,f1,0.036923,0.072072,0.092754,0.145890
5,specificity,0.997147,0.996861,0.995720,0.699957
6,fp_rate,0.002853,0.003139,0.004280,0.300043
7,g_mean,0.141455,0.200020,0.230830,0.665168


In [29]:
X_res1,y_res1=wgan_gp_shap_weighted(X_train,y_train)

[LightGBM] [Info] Number of positive: 697, number of negative: 16355
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002363 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3262
[LightGBM] [Info] Number of data points in the train set: 17052, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.040875 -> initscore=-3.155504
[LightGBM] [Info] Start training from score -3.155504


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\shap\explainers\_tree.py:583: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_3364\355030566.py:99: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_tr = y_train.ravel()


Shap Calculated.


Epoch 0 | Gen Loss: -0.4221, Critic Loss: 1.4880


Epoch 1 | Gen Loss: -1.6092, Critic Loss: 0.1024


...

Epoch 195 | Gen Loss: 1.3854, Critic Loss: -1.1984


Epoch 196 | Gen Loss: 1.4555, Critic Loss: -1.3941


Epoch 197 | Gen Loss: 0.4968, Critic Loss: -1.2196


Epoch 198 | Gen Loss: 1.4029, Critic Loss: -1.6649


Epoch 199 | Gen Loss: 1.7616, Critic Loss: -1.2668


In [30]:
for i in range(1):
    rnew=callf1(X_res1,y_res1.ravel(),X_test,y_test.ravel())

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_3364\1579888781.py:2: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rnew=callf1(X_res1,y_res1.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 16355, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5865
[LightGBM] [Info] Number of data points in the train set: 32710, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.75 MB) transferred to GPU in 0.004355 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:41:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:41:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:41:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [31]:
rnew

,metric,LightGBM,XGBoost_RF,XGBoost_GBDT,LogisticRegression
0,accuracy,0.957581,0.957991,0.957307,0.687192
1,roc_auc,0.756492,0.742386,0.734281,0.720645
2,pr_auc,0.175864,0.186113,0.163254,0.137946
3,recall,0.026756,0.043478,0.066890,0.645485
4,f1,0.049080,0.078078,0.113636,0.144461
5,specificity,0.997289,0.997004,0.995292,0.688971
6,fp_rate,0.002711,0.002996,0.004708,0.311029
7,g_mean,0.163350,0.208202,0.258021,0.666874


In [34]:
enn = EditedNearestNeighbours()
X_out, y_out = enn.fit_resample(X_res1,y_res1)

In [35]:
for i in range(1):
    rnew1=callf1(X_out, y_out.ravel(),X_test,y_test.ravel())

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_3364\900170529.py:2: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rnew1=callf1(X_out, y_out.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 11089, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5864
[LightGBM] [Info] Number of data points in the train set: 27444, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.63 MB) transferred to GPU in 0.005969 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.404059 -> initscore=-0.388580
[LightGBM] [Info] Start training from score -0.388580


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:42:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:42:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:43:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [36]:
rnew1

,metric,LightGBM,XGBoost_RF,XGBoost_GBDT,LogisticRegression
0,accuracy,0.959223,0.959496,0.958675,0.868774
1,roc_auc,0.745841,0.727655,0.724501,0.719789
2,pr_auc,0.167455,0.155694,0.144375,0.145463
3,recall,0.013378,0.013378,0.013378,0.478261
4,f1,0.026144,0.026316,0.025806,0.229719
5,specificity,0.999572,0.999857,0.999001,0.885433
6,fp_rate,0.000428,0.000143,0.000999,0.114567
7,g_mean,0.115638,0.115655,0.115605,0.650744


# 2. Modelling with LightGBM algorithm with  using SMOTE-GAN hybrids to oversample train data 


In [10]:
from gan_smoteenn import gan_augment_with_enn
from cgan_smoteenn import cgan_augment_with_enn
from wgan_smoteenn import wgan_gp_augment_with_enn

from smotified_gan_tomek import gan_augment_with_existing_resampling
from smotified_cgan_tomek import cgan_augment_with_existing_resampling
from smotified_wgan_tomek import wgan_augment_with_existing_resampling


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
Xn_2, yn_2 = gan_augment_with_enn(X_train, y_train ,apply_enn=True) #EN İYİ ENN İLE SONUÇ VERDİ
Xn_cgan_2, yn_cgan_2 = cgan_augment_with_enn(X_train, y_train,apply_enn=True)
Xn_wgan_2, yn_wgan_2 = wgan_gp_augment_with_enn(X_train, y_train,apply_enn=True)
sum=0
for i in range(1):
    r2=callf1(Xn_2, yn_2.ravel(),X_test,y_test.ravel())
    r3=callf1(Xn_cgan_2, yn_cgan_2.ravel(),X_test,y_test.ravel())
    r4=callf1(Xn_wgan_2, yn_wgan_2.ravel(),X_test,y_test.ravel())

C:\Users\Onur Yaman\Tez\Data\gan_smoteenn.py:85: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_tr = y_train.ravel()
                                                                                                                       

Epoch 0 | Gen Loss: 0.6839, Disc Loss: 0.6986


Epoch 1 | Gen Loss: 0.6877, Disc Loss: 0.6906


Epoch 2 | Gen Loss: 0.6917, Disc Loss: 0.6865


Epoch 3 | Gen Loss: 0.6957, Disc Loss: 0.6821


...

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_3364\2713485070.py:6: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r2=callf1(Xn_2, yn_2.ravel(),X_test,y_test.ravel())
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:23:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:23:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 


[LightGBM] [Info] Number of positive: 2091, number of negative: 14721
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5812
[LightGBM] [Info] Number of data points in the train set: 16812, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.38 MB) transferred to GPU in 0.003988 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.124375 -> initscore=-1.951633
[LightGBM] [Info] Start training from score -1.951633


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:24:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:24:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:24:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

[LightGBM] [Info] Number of positive: 2091, number of negative: 14325
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5812
[LightGBM] [Info] Number of data points in the train set: 16416, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.38 MB) transferred to GPU in 0.004113 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.127376 -> initscore=-1.924364
[LightGBM] [Info] Start training from score -1.924364


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:24:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:24:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:25:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [12]:
Xn, yn = gan_augment_with_existing_resampling(X_train, y_train,X_train_resampled, y_train_resampled ,apply_tomek=True)
Xn_cgan, yn_cgan = cgan_augment_with_existing_resampling(X_train, y_train,X_train_resampled, y_train_resampled,apply_tomek=True)
Xn_wgan, yn_wgan = wgan_augment_with_existing_resampling(X_train, y_train,X_train_resampled, y_train_resampled,apply_tomek=True)

C:\Users\Onur Yaman\Tez\Data\smotified_gan_tomek.py:85: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_tr = y_train.ravel()
 50%|██████████████████████████████████████████                                          | 3/6 [00:00<00:00,  4.97it/s]

Epoch 0, Step 1 | Gen Loss: 1.3890339732170105, Disc Loss: 1.3874770402908325
Epoch 0, Step 2 | Gen Loss: 0.6951680779457092, Disc Loss: 0.6924701929092407


 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [00:01<00:00,  4.99it/s]

Epoch 0, Step 3 | Gen Loss: 0.6956582069396973, Disc Loss: 0.6910614967346191
Epoch 0, Step 4 | Gen Loss: 0.6961326599121094, Disc Loss: 0.6901769638061523


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.96it/s]


...

Epoch 197 | Gen Loss: 4.4905, Critic Loss: -1.0112


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 17.63it/s]


Epoch 198 | Gen Loss: 4.0861, Critic Loss: -1.1617


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 17.74it/s]


Epoch 199 | Gen Loss: 3.6353, Critic Loss: -1.4283


In [14]:
sum=0
for i in range(1):
    r2smote=callf1(Xn, yn.ravel(),X_test,y_test.ravel())
    r3smote=callf1(Xn_cgan, yn_cgan.ravel(),X_test,y_test.ravel())
    r4smote=callf1(Xn_wgan, yn_wgan.ravel(),X_test,y_test.ravel())

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_3364\3145017709.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r2smote=callf1(Xn, yn.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 16078, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5865
[LightGBM] [Info] Number of data points in the train set: 32433, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.74 MB) transferred to GPU in 0.003982 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495730 -> initscore=-0.017082
[LightGBM] [Info] Start training from score -0.017082


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:23:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:23:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:24:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

[LightGBM] [Info] Number of positive: 16078, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5865
[LightGBM] [Info] Number of data points in the train set: 32433, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.74 MB) transferred to GPU in 0.003555 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.495730 -> initscore=-0.017082
[LightGBM] [Info] Start training from score -0.017082


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:24:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:24:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:24:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

[LightGBM] [Info] Number of positive: 15918, number of negative: 16355
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5865
[LightGBM] [Info] Number of data points in the train set: 32273, number of used features: 23
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.74 MB) transferred to GPU in 0.003622 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493230 -> initscore=-0.027083
[LightGBM] [Info] Start training from score -0.027083


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:24:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:24:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:25:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [15]:
df_merged=pd.concat([rson[['metric','LightGBM']],r['LightGBM'],r2['LightGBM'],r3['LightGBM'],r4['LightGBM'],r2smote['LightGBM'],r3smote['LightGBM'],r4smote['LightGBM']],axis=1)

In [16]:
df_merged.columns=['Metric','No SMOTE','Basic SMOTE','GAN-ENN Hybrid','CGAN-ENN Hybrid','WGAN_GP-ENN Hybrid','SMOTE-GAN-TOMEK Hybrid','SMOTE-CGAN-TOMEK Hybrid','SMOTE-WGAN_GP-TOMEK Hybrid']

In [17]:
df_merged

,Metric,No SMOTE,Basic SMOTE,GAN-ENN Hybrid,CGAN-ENN Hybrid,WGAN_GP-ENN Hybrid,SMOTE-GAN-TOMEK Hybrid,SMOTE-CGAN-TOMEK Hybrid,SMOTE-WGAN_GP-TOMEK Hybrid
0,accuracy,0.956486,0.952928,0.951834,0.951834,0.946360,0.958402,0.958265,0.957854
1,roc_auc,0.756852,0.726946,0.768532,0.776409,0.762851,0.750086,0.754174,0.761955
2,pr_auc,0.169115,0.176498,0.186039,0.187565,0.186099,0.175518,0.187800,0.173589
3,recall,0.033445,0.150502,0.204013,0.177258,0.200669,0.016722,0.010033,0.013378
4,f1,0.059172,0.207373,0.257384,0.231441,0.234375,0.031847,0.019293,0.025316
5,specificity,0.995862,0.987159,0.983735,0.984877,0.978171,0.998573,0.998716,0.998145
6,fp_rate,0.004138,0.012841,0.016265,0.015123,0.021829,0.001427,0.001284,0.001855
7,g_mean,0.182501,0.385447,0.447990,0.417824,0.443045,0.129223,0.100103,0.115556


## 3.  Modelling with LightGBM algorithm with SMOTEENN with adaptive distance weight variants 

In [ ]:
from adaptive_smoteenn import smoteenn_with_weighted_voting
distance_algos=['gaussian','inverse','exponential','rank','adaptive']
results=[]
for algo in distance_algos:
    X_resampled, y_resampled = smoteenn_with_weighted_voting(
        X_train, y_train,
        strategy=str(algo),          # 'inverse', 'exponential', 'rank', 'adaptive'
        n_neighbors_smote=5,
        n_neighbors_enn=5,
        alpha=0.75,
        beta=0.75,
        sigma=1.25,                    # for gaussian weighting
        random_state=42
    )
    
    sum=0
    
    for i in range(1):
        
        r=callf1(X_resampled, y_resampled.ravel(),X_test,y_test.ravel())

    results.append(r)


In [ ]:
final_results.columns='SMOTE_ENN_'+final_results.columns

In [ ]:
final_results.reset_index(drop=True,inplace=True)

In [ ]:
df_all=pd.concat([df_merged,final_results],axis=1)